## Read and Save Query Status in CSV for all Workflows

<br>

**Read all the JSON files for all the workflows and print out the messages and query status to a CSV file**

<br>

In [1]:
# import all the modules. NB: submit_run_ars_modules contains all the modules to submit job to ARAX

import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
#from submit_run_ars_modules import submit_to_ars, submit_to_devars, printjson, retrieve_devars_results
import glob 
import os
from collections import defaultdict
import pandas as pd
from time import sleep
from os import path
from datetime import datetime



In [2]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [3]:
def submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

##https://ars.ci.transltr.io/ars/api

def retrieve_ars_results(mid, name, check_sheet, ars_url='https://ars-dev.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    dictionary_2 = {}
    dict3 = {}
    for child in j['children']:
        print(child['status'])
        error_code = child['code']
        
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                    
                    
                    
                if name in list(check_sheet.Workflow):
                    print(name)
                    dfy = check_sheet[check_sheet['Workflow']== name]
            
                    dfy.reset_index(drop=True)

                    for index,curie_id in enumerate(dfy.Curie):
                        print(index,curie_id)
                        node_num = dfy.iloc[index][3]
                        query_id = curie_id
                        if np.isnan(dfy.iloc[index][2]) == True:
                            len_check = len(child_response['fields']['data']['message']['results'])
                        else:
                            len_check = dfy.iloc[index][2]

                        #print(node_num, query_id, len_check)

                        locs = []
                        for x, val in enumerate(child_response['fields']['data']['message']['results']):
                            #print(val)

                            if x < len_check:

                                if query_id in val['node_bindings'][node_num][0]['id']:
                                    locs.append(x)
                        if not locs:
                            check_result = f'False'
                            print(check_result)
                            #pass
                        else:
                            check_result = f'True'
                            print('curie id:', query_id, ': INCLUDED at postion N ==', locs, 'on', node_num)

                        dict3[curie_id] = check_result    
                    
                if child_response['fields']['data']['message']['knowledge_graph']['edges']:
                    if child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                            edge_ex = child_response['fields']['data']['message']['knowledge_graph']['edges']
                            test_att_values =[]
                            for val in child_response['fields']['data']['message']['knowledge_graph']['edges'].keys():
                                #print(val)
                                
                                for tx in edge_ex[val]['attributes']:
                                    if (tx['attribute_type_id'] == 'biolink:primary_knowledge_source') or (tx['attribute_type_id'] == 'biolink:original_knowledge_source') or (tx['attribute_type_id'] == 'biolink:aggregator_knowledge_source') :
                                        
                                        
                                        value_att = tx['value']
                        
                                        test_att_values.append(value_att)
                                        test_att = set(flatten_list(test_att_values))
                                        
                                        
                                        dictionary_2[child['actor']['agent']] = test_att
                    #else:
                        #dictionary_2[child['actor']['agent']] = [] 
                #else:
                   # dictionary_2[child['actor']['agent']] = []
            
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
                
            
        
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
                #dictionary_2[child['actor']['agent']] = []
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
                #dictionary_2[child['actor']['agent']] = []
        
        
        else:
            nresults = 0
            #dictionary_2[child['actor']['agent']] = []
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error' ': ' + str(error_code)
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error' ': ' + str(error_code)
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' ': ' + str(error_code) + str(dict3)
        elif ((child['status'] == 'Unknown') & (nresults == 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Unknown' ': ' + str(error_code)
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return [dictionary, dictionary_2]


#def submit_to_devars(m):
#    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

#def retrieve_devars_results(m):
#     return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))
    
def make_hyperlink(value):
    return '=HYPERLINK("%s", "%s")' % (value.format(value), value)

<br>

**The below code reads each JSON files from the Workflows A through D (subdirectories). The queries are submitted to ARAX and output is saved in a dictionary, where the key is the file name of the JSON to denote which query is being run and the values assigned to the key is the query id**

<br>


In [5]:
check_sheet = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/Query results to include or exclude - Sheet1.csv", header=0)

In [6]:
PATH = r'/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo'
EXT = "*.json"
dict_workflows = {}
for root, dirs, files in os.walk(PATH): # step 1: accessing file
    #print(root)
    for name in files:
        
        if name.endswith((".json")):
            if (name == 'C.2c_Natalizumab_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2b_Etanercept_MultSclerosis_GeneSet_and_SmallMolecule.json') or (name == 'C.2a_Imatinib_MultSclerosis_GeneSet_and_SmallMolecule.json'):
                pass
            else:
                file_read = path.join(root, name)
                dir_name = (os.path.splitext(os.path.basename(root))[0])
                print(file_read)

                filename = (os.path.splitext(os.path.basename(file_read))[0])
                print(filename)
                with open(file_read,'r') as inf:
                    query = json.load(inf)

                    kcresult = submit_to_ars(query)

                    sleep(400)

                    result_status = retrieve_ars_results(kcresult, name, check_sheet)


                    dict_workflows[filename] = kcresult

                    sleep(100)

/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.3_KCNMA1.json
A.3_KCNMA1
https://arax.ncats.io/?source=ARS&id=78d03947-b981-499b-897c-9d768a105ed2
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 168
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.2a_RHOBTB2_twohop.json
A.2a_RHOBTB2_twohop
https://arax.ncats.io/?source=ARS&id=54b09ade-0ca8-4d88-bb70-9c5c13d817eb
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 0
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.8_EGFR_simple.json
A.8_EGFR_simple
https://arax.ncats.io/?source=ARS&id=37fcdc77-addf-4a0f-a996-4c20a0240250
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 406
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowA/A.Multiomics_BigGIM_DR_KP_RHOBTB2.json
A.Multiomics_BigGIM_DR_KP_RHOBTB2
https://arax.ncats.io/?source=ARS&id=314f01ba-5eae-44fd-b0c9-d0e40bd0

https://arax.ncats.io/?source=ARS&id=a832b1e6-a190-4282-978e-0b51e1a9ae15
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 0
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin.json
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
https://arax.ncats.io/?source=ARS&id=16145c18-da37-40c0-b6ec-364f9db86f87
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 500
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin.json
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
https://arax.ncats.io/?source=ARS&id=5e2e6d2e-6ce6-41ff-b193-965e65393b0e
Done
Done
ara-arax Done 0
Done
ara-aragorn ARS Error 0
Unknown
ara-bte Unknown 0
/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowD/D.1_parkinsons-crohns.json
D.1_parkinsons-crohns
https://arax.ncats.io/?source=ARS&id=b4ec3491-45fd-4

<br>

### Codes below are for recording messages and generating outout as csv

<br>

In [7]:
dict_workflows

{'A.3_KCNMA1': '78d03947-b981-499b-897c-9d768a105ed2',
 'A.2a_RHOBTB2_twohop': '54b09ade-0ca8-4d88-bb70-9c5c13d817eb',
 'A.8_EGFR_simple': '37fcdc77-addf-4a0f-a996-4c20a0240250',
 'A.Multiomics_BigGIM_DR_KP_RHOBTB2': '314f01ba-5eae-44fd-b0c9-d0e40bd0b3b9',
 'A.0_RHOBTB2_direct': '804093ca-60c5-4f98-8691-dc645237c70c',
 'A.2_RHOBTB2_twohop': '4cd22fe5-73c3-4566-afaa-a3fd9751aa05',
 'A.Multiomics_BigGIM_DR_KP_EGFR': 'a5fdbbea-1bb1-4a4f-b7ac-ab9e17ee7d41',
 'A.2_RHOBTB2_twohop_constrained': '427f8ff6-7b29-4ee1-b0f7-7b85d7648a58',
 'A.9_EGFR_advanced': 'f087d94f-9571-47d9-a94e-3821219277ad',
 'A.2a_expanded_RHOBTB2_twohop_constrained': '8ee6223b-7899-4dd3-981b-07f4420f41f4',
 'A.1_RHOBTB2': '2420db89-e9be-49e5-a8f5-e76ea7b7dd53',
 'A.2a_expanded_RHOBTB2_twohop': '1fee0565-02a1-4c88-98f6-c6dbeef8ca26',
 'B.4e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate': 'a385228c-4533-4c50-9523-6aa8fe69d26a',
 'B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone': 'c62fb56d-0a70-4b75-b1b9-348a

In [9]:
workflow_result_messages = {}
for keys, val in dict_workflows.items():
    name = keys + '.json'
    print(name, val)
    
    result_status = retrieve_ars_results(val, name, check_sheet)
    
    workflow_result_messages[keys] = result_status

A.3_KCNMA1.json 78d03947-b981-499b-897c-9d768a105ed2
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 168
Unknown
ara-bte Unknown 0
A.2a_RHOBTB2_twohop.json 54b09ade-0ca8-4d88-bb70-9c5c13d817eb
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 0
Unknown
ara-bte Unknown 0
A.8_EGFR_simple.json 37fcdc77-addf-4a0f-a996-4c20a0240250
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 406
Unknown
ara-bte Unknown 0
A.Multiomics_BigGIM_DR_KP_RHOBTB2.json 314f01ba-5eae-44fd-b0c9-d0e40bd0b3b9
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 3
Unknown
ara-bte Unknown 0
A.0_RHOBTB2_direct.json 804093ca-60c5-4f98-8691-dc645237c70c
Done
Done
A.0_RHOBTB2_direct.json
0 PUBCHEM.COMPOUND:2950270
ara-arax ARS Error 0
Done
ara-aragorn ARS Error 0
Unknown
ara-bte Unknown 0
A.2_RHOBTB2_twohop.json 4cd22fe5-73c3-4566-afaa-a3fd9751aa05
Done
Done
ara-aragorn ARS Error 0
Done
ara-arax Done 500
Unknown
ara-bte Unknown 0
A.Multiomics_BigGIM_DR_KP_EGFR.json a5fdbbea-1bb1-4a4f-b7ac-ab9e17ee7d41
Done
Don

### Creating dataframe for workflows with PK

<br>

In [10]:
## Convert mesages to a dataframe
col = []
final_dict = defaultdict(list)

for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    for key, value in workflow_result_messages[k][0].items():
        #print(key, value)
#         if key.startswith('kp-'):
#             key_mod = key.replace('kp-','')
#         else:
#             key_mod = key
        
        final_dict[key].append(value)

    final_dict = dict(final_dict)
    
df = pd.DataFrame(final_dict).T
df.rename(columns=dict(zip(df.columns, col)), inplace=True)

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [11]:
df.replace('ARS Error', 'No Results', regex=True,inplace=True)

In [12]:
df

,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
pk_id,https://arax.ncats.io/?source=ARS&id=804093ca-...,https://arax.ncats.io/?source=ARS&id=2420db89-...,https://arax.ncats.io/?source=ARS&id=4cd22fe5-...,https://arax.ncats.io/?source=ARS&id=427f8ff6-...,https://arax.ncats.io/?source=ARS&id=54b09ade-...,https://arax.ncats.io/?source=ARS&id=1fee0565-...,https://arax.ncats.io/?source=ARS&id=8ee6223b-...,https://arax.ncats.io/?source=ARS&id=78d03947-...,https://arax.ncats.io/?source=ARS&id=37fcdc77-...,https://arax.ncats.io/?source=ARS&id=f087d94f-...,...,https://arax.ncats.io/?source=ARS&id=ac2221b1-...,https://arax.ncats.io/?source=ARS&id=d9595f13-...,https://arax.ncats.io/?source=ARS&id=c8a72b69-...,https://arax.ncats.io/?source=ARS&id=39922ddd-...,https://arax.ncats.io/?source=ARS&id=c38571e9-...,https://arax.ncats.io/?source=ARS&id=b4ec3491-...,https://arax.ncats.io/?source=ARS&id=7f738aa2-...,https://arax.ncats.io/?source=ARS&id=4bc0e00c-...,https://arax.ncats.io/?source=ARS&id=443c5c3a-...,https://arax.ncats.io/?source=ARS&id=16cdcff7-...
ara-arax,No Results: 200,Results: 200{},Results: 200{},Results: 200{},No Results: 200,No Results: 200,Results: 200{},Results: 200{},Results: 200{},Results: 200{},...,Results: 200{},Results: 200{},No Results: 200,Results: 200{},Results: 200{},No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-aragorn,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503


In [15]:
df.replace('{}','',regex=True,inplace=True)

### Creating second table with edge attribute source

<br>

In [16]:
final_dict2 = defaultdict(dict)
for k in sorted(workflow_result_messages):
    print(k)
    col.append(k)
    
    count = 0
    
    for key, value in workflow_result_messages[k][1].items():
        final_dict2[k][key] = value

A.0_RHOBTB2_direct
A.1_RHOBTB2
A.2_RHOBTB2_twohop
A.2_RHOBTB2_twohop_constrained
A.2a_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop
A.2a_expanded_RHOBTB2_twohop_constrained
A.3_KCNMA1
A.8_EGFR_simple
A.9_EGFR_advanced
A.Multiomics_BigGIM_DR_KP_EGFR
A.Multiomics_BigGIM_DR_KP_RHOBTB2
B.1_DILI-three-hop-related-to
B.2_DILI-three-hop-interacts-with
B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.3b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.3c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.3d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.3e_DILI-fourth-one-hop-from-RXNORM_40068_DextranSulfate
B.3f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone
B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone
B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone
B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol
B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin
B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin
B.4e_DILI-fourth-one-hop-

In [17]:
final_dict2

defaultdict(dict,
            {'A.1_RHOBTB2': {'ara-arax': {'infores:arax', 'infores:rtx-kg2'}},
             'A.2_RHOBTB2_twohop': {'ara-arax': {'infores:arax',
               'infores:automat-drug-central',
               'infores:automat-hmdb',
               'infores:chembl',
               'infores:connections-hypothesis',
               'infores:ctd',
               'infores:drugbank',
               'infores:drugcentral',
               'infores:hmdb',
               'infores:molepro',
               'infores:mychem-info',
               'infores:rtx-kg2',
               'infores:translator-biothings-explorer'}},
             'A.2_RHOBTB2_twohop_constrained': {'ara-arax': {'infores:arax',
               'infores:connections-hypothesis',
               'infores:drugbank',
               'infores:molepro',
               'infores:rtx-kg2'}},
             'A.2a_expanded_RHOBTB2_twohop_constrained': {'ara-arax': {'infores:arax',
               'infores:automat-hmdb',
               

In [18]:
final_dictassemble = []
for k, vs in final_dict2.items():
    #print(k,vs)
    for kv, v in vs.items():
        for t in v:
            final_dictassemble.append([k,kv,t])

In [19]:
final_dictassemble

[['A.1_RHOBTB2', 'ara-arax', 'infores:arax'],
 ['A.1_RHOBTB2', 'ara-arax', 'infores:rtx-kg2'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:automat-hmdb'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:arax'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:mychem-info'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:chembl'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:translator-biothings-explorer'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:ctd'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:drugcentral'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:molepro'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:connections-hypothesis'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:drugbank'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:automat-drug-central'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:hmdb'],
 ['A.2_RHOBTB2_twohop', 'ara-arax', 'infores:rtx-kg2'],
 ['A.2_RHOBTB2_twohop_constrained', 'ara-arax', 'infores:arax'],
 ['A.2_RHOBTB2_twohop_constrained', 'ara-arax', 'infores:mo

In [20]:
column_names = ['Workflow', 'ARS-KPs', 'Values']
df2 = pd.DataFrame(final_dictassemble, columns=column_names)
df2 = df2.astype(str)

In [21]:
df2.Values = df2.Values.apply(lambda x: x[2:-2] if ('[' in x) else x)

In [22]:
df2test = df2.groupby(['Workflow','Values'])['ARS-KPs'].agg(list)

In [23]:
df2test = pd.DataFrame(df2test.unstack().T)

In [24]:
df2test

Workflow,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine
Values,,,,,,,,,,,,,,,,,,,
icees-dili,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax]
infores:automat-ctd,NaN,NaN,NaN,NaN,NaN,[ara-arax],[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-drug-central,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-gtopdb,NaN,NaN,NaN,NaN,NaN,[ara-arax],[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:automat-hmdb,NaN,[ara-arax],NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:biothings-multiomics-biggim-drugresponse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
infores:biothings-multiomics-clinical-risk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN
infores:chembl,NaN,[ara-arax],NaN,NaN,[ara-arax],NaN,NaN,NaN,NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN,[ara-arax],NaN,NaN,NaN


In [25]:
#df2test.drop([''], axis=0, inplace=True)

In [26]:
#df2test.index = df2test.index.map(lambda x: x[2:-2] if ('[' in x) else x)

In [27]:
#df2.replace([], 'None', regex=True,inplace=True)

In [28]:
#df2test = df2test.mask(df2test.applymap(type).eq(list) & ~df2test.astype(bool))

In [29]:
df2test = df2test.rename_axis(None)

In [30]:
df2test.columns.name = None

In [31]:
df2test.fillna('', inplace=True)

In [32]:
df2test

,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_RHOBTB2,B.2_DILI-three-hop-interacts-with,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine
icees-dili,,,,,,,,,[ara-arax],,,,,,,,,,
infores:arax,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax]
infores:automat-ctd,,,,,,[ara-arax],[ara-arax],,,,,,,,,,,,
infores:automat-drug-central,,[ara-arax],,,,,,,,,,,,,,,,,
infores:automat-gtopdb,,,,,,[ara-arax],[ara-arax],,,,,,,,,,,,
infores:automat-hmdb,,[ara-arax],,[ara-arax],,,,,,,,,,,,,,,
infores:biothings-multiomics-biggim-drugresponse,,,,,,,,[ara-arax],,,,,,,,,,,
infores:biothings-multiomics-clinical-risk,,,,,,,,,[ara-arax],,,,,,,,,[ara-arax],
infores:chembl,,[ara-arax],,,[ara-arax],,,,,,,[ara-arax],,,,[ara-arax],,,
infores:cohd,,,,,,,,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],,,,,[ara-arax],[ara-arax],


In [33]:
infores_catalog = pd.read_csv("/Users/priyash/Documents/GitHub/minihackathons/Notebooks/InfoRes Catalog - Translator InfoRes Catalog.csv", header=1)

In [34]:
infores_catalog = infores_catalog[['id', 'name','translator category','has contributor']]
infores_catalog = infores_catalog[:335]
dict_map = {}
for i in df2test.index.values:
    if i in infores_catalog['id'].values:
        indices = infores_catalog[infores_catalog['id']==i].index[0]
        if pd.notnull(infores_catalog.iloc[indices]['has contributor']):
            dict_map[i] = infores_catalog.iloc[indices]['translator category']
        else:
            dict_map[i] = 'External Source'
    else:
        dict_map[i] = 'Illegal value'

        
        
        
        
        
        
df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog']=df2test.index.map(dict_map)
df2test['sort']=pd.Categorical(df2test['Translator_Category_Complaint_to_ColL&M_InforesCatalog'], ["KP", "ARA",'External Source', 'Illegal Value'])
df2test =df2test.sort_values(['sort'])
df2test  = df2test.rename_axis(None)
cols = df2test.columns.tolist()
cols = [cols[-2]] + cols[:-2]
df2test = df2test[cols]
df2test['Query Type'] = 'Async'

df2test = df2test[['Query Type']+ list(df2test.columns[:-1])]

In [35]:
df2test

,Query Type,Translator_Category_Complaint_to_ColL&M_InforesCatalog,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,A.9_EGFR_advanced,A.Multiomics_BigGIM_DR_KP_RHOBTB2,...,B.3a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.3g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.4a_DILI-fourth-one-hop-from-CHEBI_41879_Dexamethasone,B.4b_DILI-fourth-one-hop-from-MESH_D000077185_Resveratrol,B.4c_DILI-fourth-one-hop-from-MESH_D000077385_Silybin,B.4d_DILI-fourth-one-hop-from-MESH_D003474_Curcumin,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine
infores:connections-hypothesis,Async,KP,,[ara-arax],[ara-arax],,[ara-arax],,,,...,,,,,,,,,[ara-arax],
infores:automat-ctd,Async,KP,,,,,,[ara-arax],[ara-arax],,...,,,,,,,,,,
infores:automat-drug-central,Async,KP,,[ara-arax],,,,,,,...,,,,,,,,,,
infores:automat-gtopdb,Async,KP,,,,,,[ara-arax],[ara-arax],,...,,,,,,,,,,
infores:automat-hmdb,Async,KP,,[ara-arax],,[ara-arax],,,,,...,,,,,,,,,,
infores:biothings-multiomics-biggim-drugresponse,Async,KP,,,,,,,,[ara-arax],...,,,,,,,,,,
infores:biothings-multiomics-clinical-risk,Async,KP,,,,,,,,,...,,,,,,,,,[ara-arax],
infores:mychem-info,Async,KP,,[ara-arax],,,,,,,...,,,,,,,,,,
infores:cohd,Async,KP,,,,,,,,,...,[ara-arax],[ara-arax],[ara-arax],,,,,[ara-arax],[ara-arax],
infores:rtx-kg2,Async,KP,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,...,,,[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],[ara-arax],,[ara-arax]


<br>

## Converting the Pk's to hyperlink

<br>

In [36]:
df.loc['pk_id'] = df.loc['pk_id'].apply(lambda x: make_hyperlink(x))

In [37]:
df.rename({'pk_id': 'pk'}, inplace=True)

In [38]:
df.sort_index(inplace=True)

In [39]:
df['Query Type'] = 'Async'

df = df[['Query Type']+ list(df.columns[:-1])]

In [40]:
df

,Query Type,A.0_RHOBTB2_direct,A.1_RHOBTB2,A.2_RHOBTB2_twohop,A.2_RHOBTB2_twohop_constrained,A.2a_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop,A.2a_expanded_RHOBTB2_twohop_constrained,A.3_KCNMA1,A.8_EGFR_simple,...,B.4f_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5877_Methylprednisone,B.4g_DILI-fourth-one-hop-from-PUBCHEM.COMPOUND_5755_Prednisone,B.5_DILI_branched-four-hop,C.1a_SmallMolecule_real_world_evidence_MultSclerosis,C.3a_MultSclerosis_related_to_Nimodipine,D.1_parkinsons-crohns,D.2_ssri-heart-disease,D.3_ssri-heart-disease-one-hop,D.4_tryptophan-kynurenine,D.6_metformin-ferritin
ara-aragorn,Async,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,...,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-arax,Async,No Results: 200,Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,Results: 200,Results: 200,Results: 200,...,Results: 200,Results: 200,No Results: 200,Results: 200,Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200,No Results: 200
ara-bte,Async,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,...,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503,Unknown: 503
pk,Async,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...",...,"=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&...","=HYPERLINK(""https://arax.ncats.io/?source=ARS&..."


In [41]:
#date = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
#wks_name = 'Workflow Progress Tracker Asynchronous_' + date

In [42]:
#wks_name

<br>

### Pushing dataframe to excel sheet on google drive

<br>


**Here I am using the google drive API to push the daatframe into an axcel sheet 
Every individula has the unique credential file that they need to create for google drive API -- 
"araxworkflowprogresstesting-2632632db8be.json" -- is the credential used from my drive. place this json file where
the ReadAndRunAllWorkFLows.ipynb will be. NB: i have removed my credntial file for privacy reasons. Always remove
the json file before making committs to the repo. To use googe Drive API follow: https://towardsdatascience.com/how-to-manage-files-in-google-drive-with-python-d26471d91ecd**

<br>

<br>

## Google Drive API

**Push the dataframe to a google sheet via google drive API and then format the google spread sheet to add hyperlink to pk's and color the cells

<br>

### Push Dataframe 1

<br>

In [43]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
from gspread_formatting import *
import gspread_dataframe as gd

In [44]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'araxworkflowprogresstesting-2632632db8be.json', scope)
gc = gspread.authorize(credentials)

In [45]:
ws = gc.open("workflow_progress_tracker").worksheet("Workflow Progress Tracker_2021_09_23-04_13_54_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

existing.loc['pk'][:] = existing.loc['pk'][:].apply(lambda x: x.replace('" "', '","'))

existing.loc['pk'][1]

updated = existing.append(df)
gd.set_with_dataframe(ws, updated,include_index=True,include_column_header=True)

In [46]:
#spreadsheet_key = '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A'
#spreadsheet_key = '1sPpBIkxrHbQNiTm5oPs9-5KrjsyXcgaVAxknJj-u8pY'
#wks_name = 'Workflow Progress Tracker_' + date
#d2g.upload(df, spreadsheet_key, "TESTING ONLY", credentials=credentials, row_names=True)

In [47]:
gc = gspread.service_account(filename='/Users/priyash/Documents/GitHub/minihackathons/Notebooks/araxworkflowprogresstesting-2632632db8be.json')
wksh = gc.open("workflow_progress_tracker")
#sh = wksh.worksheet(wks_name)
sh = wksh.worksheet("Workflow Progress Tracker_2021_09_23-04_13_54_AM")

In [48]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(1,0,0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

In [49]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.5, 0.0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 1}}}}}}},
  {},
  {}]}

In [50]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['No Results']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0.75, 0))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.49803922}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Error'}]},
      'format': {'backgroundColor': {'red': 1},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'r

In [51]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['ARS Error']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.0, 0.75, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'red': 0.7490196,
         'green': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'Results'}]},
      'format': {'backgroundColor': {'green': 0.49803922},
       'textFormat': {'bo

In [52]:
rule = ConditionalFormatRule(
    ranges=[GridRange.from_a1_range('C2:{}23', sh)],
    booleanRule=BooleanRule(
        condition=BooleanCondition('TEXT_STARTS_WITH', ['Unknown']),
        format=CellFormat(textFormat=textFormat(bold=True), backgroundColor=Color(0.75, 0, 0.75))
    )
)
rules = get_conditional_format_rules(sh)
rules.append(rule)
rules.save()

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'ARS Error'}]},
      'format': {'backgroundColor': {'green': 0.7490196, 'blue': 0.7490196},
       'textFormat': {'bold': True},
       'backgroundColorStyle': {'rgbColor': {'green': 0.7490196,
         'blue': 0.7490196}}}}}}},
  {'deleteConditionalFormatRule': {'rule': {'ranges': [{'sheetId': 1650299279,
       'startRowIndex': 1,
       'endRowIndex': 1000,
       'startColumnIndex': 2,
       'endColumnIndex': 100}],
     'booleanRule': {'condition': {'type': 'TEXT_STARTS_WITH',
       'values': [{'userEnteredValue': 'No Results'}]},
      'format': {'backgroundColor': {'red': 0.7490196, 'green': 0.7490196},
    

In [54]:
# Select a range
cell_list = sh.range('B19:AS19')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_28-08_13_54_AM'!B19:AS19",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [55]:
# Select a range
cell_list = sh.range('B23:AS23')

# Update in batch
sh.update_cells(cell_list,value_input_option='USER_ENTERED')


{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'updatedRange': "'Workflow Progress Tracker_2021_09_28-08_13_54_AM'!B23:AS23",
 'updatedRows': 1,
 'updatedColumns': 44,
 'updatedCells': 44}

In [56]:
set_frozen(sh, cols=1, rows=1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}

### Push Dataframe 2

<br>

In [57]:
ws2 = gc.open("workflow_progress_tracker").worksheet("edge_attribute_source_2021_09_28-08_13_54_AM")

#existing = gd.get_as_dataframe(ws)
existing = pd.DataFrame(ws2.get_all_records())

existing.set_index('', inplace=True)



existing = existing.rename_axis(None)

updated = existing.append(df2test)
gd.set_with_dataframe(ws2, updated,include_index=True,include_column_header=True)


In [58]:
set_frozen(ws2, cols=1, rows =1)

{'spreadsheetId': '1O1cMmYGxoIqP6xbzj6FG5owiKQVg57wx2O_XIA_hN_A',
 'replies': [{}]}